# Tests

In [1]:
import pandas as pd
import os
import numpy as np
import biotite.structure.io.pdb as pdb
import biotite.structure as struc
from dotenv import load_dotenv

from dotenv import load_dotenv
load_dotenv("../../.env")
DATA_TABLE = os.getenv("TABLE")
POSITIVE_CONTROL = os.getenv("POSITIVE_CONTROL")
OUTPUT_TABLE = "output/pdb_biochem_properties.csv"
# ring atoms
AROMATICS = {"PHE": ["CG", "CD1", "CD2", "CE1", "CE2", "CZ"], 
             "TYR": ["CG", "CD1", "CD2", "CE1", "CE2", "CZ"], 
             "TRP": ["CD2", "CE2", "CE3", "CZ3", "CH2", "CZ2", "NE1"]}

In [3]:
def surroundingAtom(source_atom:np.array, structure_array:list, threshold:float) -> list:
    """

        Find structure atoms within trehold distance in amstrongs from given atom

        PARAMETERS
        ----------
        source_atom: np.array:
        structure_array:list
        treshold:float

        RETURN
        ------
        resi_below_treshold:list: list of residues below given treshold
        
    """
    distances = struc.distance(source_atom, structure_array)
    structure_array.add_annotation("distance", dtype=float)
    structure_array.distance = distances
    resi_below_threshold = [atom for atom in structure_array if atom.distance < threshold]
    if resi_below_threshold:
        resi_below_threshold = struc.array(resi_below_threshold)
    return resi_below_threshold

In [4]:
df = pd.read_csv(DATA_TABLE)
df = df[df["Chain"].isna() == False]
#path_df = pd.read_csv(PATHS_TABLE, names=["structure_path"]).drop_duplicates()
#path_df["PDB code"] = path_df["structure_path"].apply(lambda x: x.split("/")[-1].split("_")[0])
#df = pd.merge(df, path_df, how="left")
df["structure_path"] = df.apply(lambda x: os.path.join(POSITIVE_CONTROL, x["PDB code"].lower()+"_"+x["Chain"]+".pdb"), axis=1)
df["match_residues"] = df.apply(lambda x: "_".join( 
                    [str(i) for i in sorted([x["Position 1\r\n(Bond 1)"], x["Position 2\r\n(catalytic)"], x["Position 3\r\n(Bond 2)"]])]
                    ), axis=1)
# Restrict to intramolecular isopeptide bond
cond1 = (df["Is bonded"] == True)
cond2 = (df["Interchain"] == False)
# Consider bad rotamers for comparison
#cond3 = (df["Bad rotamer"] == False)
df = df[cond1 & cond2]

In [5]:
dist_threshold=10
# Cutoffs for aro cap
# Adopt some approximate values since I am not considering hydrogens
# max pi-H distance + C-H distance
dist_cutoff = 4.3+1.09
angle_cutoffs = [50, 125]

outlist = []
for index, row in df.iloc[:1].iterrows():
    struct_path = row["structure_path"]
    chain = row["Chain"]
    r1 = row["Position 1\r\n(Bond 1)"]
    pdb_file = pdb.PDBFile.read(struct_path)
    # Exclude water and consider the right chain
    structure = struc.array([atom for atom in pdb_file.get_structure()[0] if atom.hetero==False and atom.chain_id == chain])

    # Atoms within dist_threshold A from lys NZ
    lys_nz = struc.array([atom for atom in structure if atom.res_id==r1 and atom.atom_name=="NZ"])
    atoms_within_threshold = surroundingAtom(lys_nz, structure, dist_threshold)
    # Get aro residues
    aro_residues_within_threshold = list(set(struc.array([atom for atom in atoms_within_threshold if atom.res_name in AROMATICS]).res_id))
    aro_rings = struc.array([atom for atom in structure if atom.res_id in aro_residues_within_threshold \
                            and atom.atom_name in AROMATICS.get(atom.res_name, [])])
    # This is necessary to exclude incomplete residues which may not have an aro ring
    aro_residues_within_threshold = list(set(aro_rings.res_id))

    # Order aro rings by distance with ring
    def get_distance(res_id):
        """Get distance with aro ring"""
        ring_atoms = struc.array([atom for atom in aro_rings if atom.res_id==res_id])
        return np.mean(ring_atoms.distance)

    aro_residues_within_threshold = sorted(aro_residues_within_threshold, key=lambda res_id: get_distance(res_id))
    
    # Iterate until a cap is detected otherwise return the first aro residue
    cap_aro = None
    first_aro = None
    for res_id in aro_residues_within_threshold:
        # If cap aro has been detected, break
        if cap_aro:
            break
        ring_atoms = struc.array([atom for atom in aro_rings if atom.res_id==res_id])
        ring_center = struc.centroid(ring_atoms)
        # Compute the vectors along the plane of the ring
        v1 = ring_atoms[1].coord - ring_atoms[0].coord  # Vector between two ring atoms
        v2 = ring_atoms[2].coord - ring_atoms[0].coord
        normal_vector = np.cross(v1, v2)
        # Normalize the normal vector to get a unit vector
        normal_vector /= np.linalg.norm(normal_vector)
        # Get the vector from the N atom (nz) to the center of the ring (tyr_ring_center)
        nz = lys_nz.coord[0]
        nz_to_ring_center = nz - ring_center
        # Normalize this vector as well
        nz_to_ring_center /= np.linalg.norm(nz_to_ring_center)
        # Calculate the dot product between the two vectors
        dot_product = np.dot(normal_vector, nz_to_ring_center)
        # Use the dot product to calculate the angle (in radians) between the vectors
        angle_rad = np.arccos(dot_product)
        # Convert the angle to degrees
        angle_deg = np.degrees(angle_rad)
        # Calculate the distance between the N atom and the ring center
        distance = struc.distance(ring_center, nz)
        if first_aro == None:
            first_aro = [struct_path, chain, r1, res_id, ring_atoms.res_name[0], distance, angle_deg]
        if distance < dist_cutoff and ((angle_deg < angle_cutoffs[0]) or (angle_deg > angle_cutoffs[1])):
            cap_aro = [struct_path, chain, r1, res_id, ring_atoms.res_name[0], distance, angle_deg]
        
    final_aro = cap_aro if cap_aro else first_aro
    outlist.append(final_aro)

In [96]:
# Save isopep and aro atoms

dist_threshold=10
# Cutoffs for aro cap
# Adopt some approximate values since I am not considering hydrogens
# max pi-H distance + C-H distance
dist_cutoff = 4.3+1.09
angle_cutoffs = [50, 125]

for index, row in df[(~df["Interchain"])&(~df["Bad rotamer"])&(df["Is bonded"])].iterrows():
    struct_path = row["structure_path"]
    chain = row["Chain"]
    pdb_code = row["PDB code"]
    r1 = row["Position 1\r\n(Bond 1)"]
    r2 = row["Position 2\r\n(catalytic)"]
    r3 = row["Position 3\r\n(Bond 2)"]
    pdb_file = pdb.PDBFile.read(struct_path)
    # Exclude water and consider the right chain
    structure = struc.array([atom for atom in pdb_file.get_structure()[0] if atom.hetero==False and atom.chain_id == chain])

    # Atoms within dist_threshold A from lys NZ
    lys_nz = struc.array([atom for atom in structure if atom.res_id==r1 and atom.atom_name=="NZ"])
    atoms_within_threshold = surroundingAtom(lys_nz, structure, dist_threshold)
    # Get aro residues
    aro_residues_within_threshold = list(set(struc.array([atom for atom in atoms_within_threshold if atom.res_name in AROMATICS]).res_id))
    aro_rings = struc.array([atom for atom in structure if atom.res_id in aro_residues_within_threshold \
                            and atom.atom_name in AROMATICS.get(atom.res_name, [])])
    # This is necessary to exclude incomplete residues which may not have an aro ring
    aro_residues_within_threshold = list(set(aro_rings.res_id))

    # Order aro rings by distance with ring
    def get_distance(res_id):
        """Get distance with aro ring"""
        ring_atoms = struc.array([atom for atom in aro_rings if atom.res_id==res_id])
        return np.mean(ring_atoms.distance)

    aro_residues_within_threshold = sorted(aro_residues_within_threshold, key=lambda res_id: get_distance(res_id))
    
    # Iterate until a cap is detected otherwise return the first aro residue
    for res_id in aro_residues_within_threshold:
        ring_atoms = struc.array([atom for atom in aro_rings if atom.res_id==res_id])
        ring_center = struc.centroid(ring_atoms)
        # Compute the vectors along the plane of the ring
        v1 = ring_atoms[1].coord - ring_atoms[0].coord  # Vector between two ring atoms
        v2 = ring_atoms[2].coord - ring_atoms[0].coord
        normal_vector = np.cross(v1, v2)
        # Normalize the normal vector to get a unit vector
        normal_vector /= np.linalg.norm(normal_vector)
        # Get the vector from the N atom (nz) to the center of the ring (tyr_ring_center)
        nz = lys_nz.coord[0]
        nz_to_ring_center = nz - ring_center
        # Normalize this vector as well
        nz_to_ring_center /= np.linalg.norm(nz_to_ring_center)
        # Calculate the dot product between the two vectors
        dot_product = np.dot(normal_vector, nz_to_ring_center)
        # Use the dot product to calculate the angle (in radians) between the vectors
        angle_rad = np.arccos(dot_product)
        # Convert the angle to degrees
        angle_deg = np.degrees(angle_rad)
        # Calculate the distance between the N atom and the ring center
        distance = struc.distance(ring_center, nz)
        outlist = []
        if distance < dist_cutoff and ((angle_deg < angle_cutoffs[0]) or (angle_deg > angle_cutoffs[1])):
            for atom in [a for a in structure if a.res_id in [r1,r2,r3]]:
                outlist.append(atom)
            for atom in ring_atoms:
                outlist.append(atom)
            struc.io.save_structure(f"tmp/{r1}_{r2}_{r3}_{pdb_code}.pdb", struc.array(outlist))
            break
            

In [8]:
ring_atoms.coord

array([[  2.278,  42.578, -12.882],
       [  3.292,  43.594, -14.616],
       [  2.13 ,  42.975, -14.235],
       [  1.213,  41.916, -12.249],
       [  0.962,  42.729, -14.96 ],
       [  0.047,  41.67 , -12.975],
       [ -0.068,  42.075, -14.314]], dtype=float32)